<a href="https://colab.research.google.com/github/Hastieiliat/Final-Project/blob/Dave/Crime_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [816 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-12 23:04:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-07-12 23:04:08 (11.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M20-Final").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://crime2021austin.s3.us-east-2.amazonaws.com/crime_updated.csv"
spark.sparkContext.addFile(url)

In [5]:
crime_df = spark.read.csv(SparkFiles.get("crime_updated.csv"), sep=",", header=True, inferSchema=True)

In [6]:
crime_df.show()

+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|Incident_Number|Highest_Offense_Description|Occurred_Date|Occurred_Time|Report_Date|Report_Time|Zip_Code|Census_Tract|Clearance_Date|X_coordinate|Y_coordinate|   Latitude|   Longitude|
+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|2.0213650197E10|       ASSAULT W/INJURY-...|   12/31/2021|          436| 12/31/2021|        436|   78736|       330.0|    12/31/2021|     3066128|     3066128|30.23640862|-97.89627861|
|2.0213650632E10|       ASSAULT W/INJURY-...|   12/31/2021|         1223| 12/31/2021|       1223|   78726|       346.0|    12/31/2021|     3081086|     3081086|30.42004712|-97.84416748|
|2.0213650144E10|       AGG ASLT STRANGLE...|   12/31/2021|          2

In [7]:
crime_df.printSchema()

root
 |-- Incident_Number: double (nullable = true)
 |-- Highest_Offense_Description: string (nullable = true)
 |-- Occurred_Date: string (nullable = true)
 |-- Occurred_Time: integer (nullable = true)
 |-- Report_Date: string (nullable = true)
 |-- Report_Time: integer (nullable = true)
 |-- Zip_Code: integer (nullable = true)
 |-- Census_Tract: double (nullable = true)
 |-- Clearance_Date: string (nullable = true)
 |-- X_coordinate: integer (nullable = true)
 |-- Y_coordinate: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [8]:
print((crime_df.count(), len(crime_df.columns)))

(43920, 13)


In [9]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
drop_na_crime_df = crime_df.dropna()
drop_na_crime_df.show()

+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|Incident_Number|Highest_Offense_Description|Occurred_Date|Occurred_Time|Report_Date|Report_Time|Zip_Code|Census_Tract|Clearance_Date|X_coordinate|Y_coordinate|   Latitude|   Longitude|
+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|2.0213650197E10|       ASSAULT W/INJURY-...|   12/31/2021|          436| 12/31/2021|        436|   78736|       330.0|    12/31/2021|     3066128|     3066128|30.23640862|-97.89627861|
|2.0213650632E10|       ASSAULT W/INJURY-...|   12/31/2021|         1223| 12/31/2021|       1223|   78726|       346.0|    12/31/2021|     3081086|     3081086|30.42004712|-97.84416748|
|2.0213650144E10|       AGG ASLT STRANGLE...|   12/31/2021|          2

In [10]:
print((drop_na_crime_df.count(), len(drop_na_crime_df.columns)))

(43920, 13)


In [11]:
# Create the crime_table. DataFrame
crime_df = drop_na_crime_df.select(["Incident_Number","Highest_Offense_Description","Occurred_Date","Occurred_Time","Report_Date","Report_Time","Zip_Code","Census_Tract","Clearance_Date","X_Coordinate","Y_Coordinate","Latitude","Longitude"])
crime_df.show()

+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|Incident_Number|Highest_Offense_Description|Occurred_Date|Occurred_Time|Report_Date|Report_Time|Zip_Code|Census_Tract|Clearance_Date|X_Coordinate|Y_Coordinate|   Latitude|   Longitude|
+---------------+---------------------------+-------------+-------------+-----------+-----------+--------+------------+--------------+------------+------------+-----------+------------+
|2.0213650197E10|       ASSAULT W/INJURY-...|   12/31/2021|          436| 12/31/2021|        436|   78736|       330.0|    12/31/2021|     3066128|     3066128|30.23640862|-97.89627861|
|2.0213650632E10|       ASSAULT W/INJURY-...|   12/31/2021|         1223| 12/31/2021|       1223|   78726|       346.0|    12/31/2021|     3081086|     3081086|30.42004712|-97.84416748|
|2.0213650144E10|       AGG ASLT STRANGLE...|   12/31/2021|          2

In [12]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://____:5432/final"
config = {"user":"postgres", 
          "password": "___", 
          "driver":"org.postgresql.Driver"}